In [1]:
import os
from dotenv import load_dotenv
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph, START
load_dotenv()



def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                " You are a helpful AI assistant, collaborating with other assistants in conducting a maritime search and rescue operation."
                " ## Following are the involve parties in the operation:"
                " Ground station operator located in the ground station responsible for aircraft tasking from ground."
                " Pilot located in the SAR cockpit, responsible for flying the aircraft and backup communication."
                " Copilot located in the SAR cockpit, responsible for communication with the ground, navigation and as a backup pilot."
                " Search operator in the SAR cockpit, responsible for operating the search radar and camera Tasks all Aircraft search asset ; Search Mission Comms with Ground; Updates situational Awareness; Search planning; Backup Navigationasks all Aircraft search asset ; Search Mission Comms with Ground; Updates situational Awareness; Search planning; Backup Navigation."
                " Use the provided tools to progress the flight."
                " Ensure to pass on the communication to the next assistant when you are done."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools, tool_choice="any")

In [2]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool


@tool
def operate_systems(operator:str, action: str, system: str, value: str):
    """
    Use this to operate the aircraft systems. 
    'operator' can be any of the following: 'pilot', 'copilot', 'search operator', or 'ground station operator'.
    'action' can be either 'query', 'operate', or 'report'.
    'system' can be any aircraft or ground control system.
    'value' is the value to set the system to, if applicable. if not applicable, set to 'n/a'.
    """
    return {"operator": operator, "action":action, "system":system, "value": value}

@tool
def radio(speaker:str, listener:str, conversation:str):
    """Use this to parse any conversation between the two speakers"""
    return {"speaker":speaker, "listener":listener, "content":conversation} 

In [3]:
import operator
import functools
from typing import Annotated, Sequence, TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage

# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }

## Role definition:

In [4]:
pilot_role = """
## You're responsible for piloting the aircraft and communicating with the Ground Control:
- Safely operate SAR aircraft in all conditions.
- Work closely with the Co-Pilot to follow the flight plan and respond to any adjustments in
the mission.
- Maintain communication with the Ground Station Operator as a backup.
- Make real-time decisions to ensure the safety of the crew and the effectiveness of the
search mission.
- Conduct the aircraft in a manner that optimizes the effectiveness of the search patterns
and strategies devised by the Search System Operator.
- Handle emergency situations according to SAR protocols.
"""

GC_ROLE= """
## You're responsible for aircraft tasking tasking from ground:
- Coordinate with maritime authorities and other SAR entities to define search areas and
priorities.
- Task SAR aircraft based on incoming alerts and information.
- Provide pilots with weather updates and any other relevant environmental information.
- Monitor SAR operations and maintain communication with SAR aircraft and other
involved parties.
- Log and update all SAR mission details in real-time.
- Coordinate rescue efforts based on findings from SAR operations.
- Ensure all SAR protocols and procedures are followed
"""

copilot_role = """
- Primary communicator between the SAR aircraft and the Ground Station, relaying
mission updates, and receiving instructions.
- Assist the Pilot in navigating and adjusting flight paths as needed.
- Serve as a liaison between the Pilot and the Search System Operator, ensuring clear
communication of search areas and objectives.
- Take over piloting duties if necessary.
- Manage the aircraft's systems and ensure all navigational instruments are functioning
properly.
- Participate in pre-flight planning and post-mission debriefs.
"""

search_operator_role = """
- Operate and manage all onboard search equipment, including radar, sonar, and cameras.
- Continuously communicate with the Ground Station to update on search progress and
receive new instructions.
- Plan and adjust the search strategy in real-time based on the analysis of search data and
changing conditions.
- Provide updates to the SAR team on situational awareness, induding potential sightings
or detections.
- Assist in navigation as a backup, ensuring the search patterns are accurately followed.
- Coordinate with the Pilot for any necessary adjustments to the flight plan based on
search findings or equipment capabilities.
"""

## Define agent nodes

In [5]:
llm = ChatOpenAI(model="gpt-4o", max_tokens=200, temperature=0.5)
#llm = llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

# Research agent and node
pilot = create_agent(
    llm,
    [radio, operate_systems],
    system_message=pilot_role,
)
pilot_node = functools.partial(agent_node, agent=pilot, name="Pilot")

# chart_generator
ground_control = create_agent(
    llm,
    [radio, operate_systems],
    system_message=GC_ROLE,
)
gc_node = functools.partial(agent_node, agent=ground_control, name="GC")

#copilot
copilot = create_agent(
    llm,
    [radio, operate_systems],
    system_message=copilot_role,
)
copilot_node = functools.partial(agent_node, agent=copilot, name="Co-Pilot")

#copilot
search_operator = create_agent(
    llm,
    [radio, operate_systems],
    system_message=search_operator_role,
)
search_operator_node = functools.partial(agent_node, agent=search_operator, name="Search Operator")

In [6]:
from langgraph.prebuilt import ToolNode

tools = [radio, operate_systems]
tool_node = ToolNode(tools)

## Edge logic

In [7]:
# Either agent can decide to end
from typing import Literal


def router(state) -> Literal["call_tool", "__end__", "continue"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "__end__"
    return "continue"

## Define graph

In [8]:
workflow = StateGraph(AgentState)

workflow.add_node("Pilot", pilot_node)
workflow.add_node("GC", gc_node)
workflow.add_node("Co-Pilot", copilot_node)
workflow.add_node("call_tool", tool_node)
workflow.add_node("Search Operator", search_operator_node)

# the params are: 'from', 'to', 'condition'
workflow.add_conditional_edges(
    "Co-Pilot",
    router,
    {"continue": "Pilot", "call_tool": "call_tool", "__end__": END},
)
workflow.add_conditional_edges(
    "GC",
    router,
    {"continue": "Co-Pilot", "call_tool": "call_tool", "__end__": END},
)
workflow.add_conditional_edges(
    "Pilot",
    router, 
    {"continue": "GC", "call_tool": "call_tool", "__end__": END},
)
workflow.add_conditional_edges(
    "Search Operator",
    router, 
    {"continue": "Pilot", "call_tool": "call_tool", "__end__": END},
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "Pilot": "Pilot",
        "GC": "GC",
        "Co-Pilot": "Co-Pilot",
        "Search Operator": "Search Operator",
    },
)
workflow.add_edge(START, "GC")
graph = workflow.compile()

## Invoke

In [25]:
events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Please role play communications of these four actors and the sim for a takeoff sequence from dead stop at the end of the runway just before takeoff to airborne and on course to the first search area.",
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 150},
)

In [26]:
idx = 1
for s in events:
    key = list(s.keys())[0]
    if key == "call_tool":
        print(f"Step{idx}: {s[key]['messages'][0].name} --> {s[key]['messages'][0].content}") 
        idx += 1
        print("----")

Step1: radio --> {"speaker": "ground station operator", "listener": "pilot", "content": "SAR Aircraft, you are clear for takeoff. Winds are calm. Proceed to the first search area as planned. Over."}
----
Step2: operate_systems --> {"operator": "pilot", "action": "operate", "system": "engine", "value": "start"}
----
Step3: operate_systems --> {"operator": "pilot", "action": "operate", "system": "throttle", "value": "set to takeoff power"}
----
Step4: radio --> {"speaker": "pilot", "listener": "copilot", "content": "Engines are started, flaps in takeoff position, and throttle set to takeoff power. Ready for takeoff checklist."}
----
Step5: operate_systems --> {"operator": "copilot", "action": "operate", "system": "instruments", "value": "check all green"}
----
Step6: radio --> {"speaker": "copilot", "listener": "pilot", "content": "Instruments are green, radios set. We are go for takeoff."}
----
Step7: operate_systems --> {"operator": "pilot", "action": "operate", "system": "brakes", "va

KeyboardInterrupt: 

In [ ]:
import os
import anthropic
from dotenv import load_dotenv
load_dotenv()

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
)

print(os.environ.get("ANTHROPIC_API_KEY"))

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content)